In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader -> datasets 라이브러리랑 충돌

c:\Users\ehddl\AppData\Local\pypoetry\Cache\virtualenvs\sns-categorizer-wO1G7-CE-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count()) 
print(torch.cuda.get_device_name(0))  

True
1
NVIDIA GeForce RTX 3050


In [ ]:
# pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121
# poetry run pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121

In [4]:
os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/sns-category-classifier")

In [5]:
data = pd.read_csv("tests/final_fine_tuning_data.csv", index_col=0)
data

,media_cn_cleaned,label_id
0,휴가 돌려죠,19
1,관종들 릴스 릴스타그램 릴스초보,21
2,날이 좋아서,19
3,행복했던 9월 고마워,19
4,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,8
...,...,...
39925,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,25
39926,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,25
39927,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,25
39928,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,25


In [6]:
train, test = train_test_split(data, test_size= 0.2, stratify=data['label_id'], random_state=42)

In [7]:
# model & toknizer loading

model_name = "kykim/bert-kor-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = BertTokenizerFast.from_pretrained(model_name)

## 모델이 bert 계열이고, 속도가 중요하면 BertTkenizerFast
## 모델을 자주 비꾸거나 여러 모델 실험할 계획이면 AutoTokenizer

In [8]:
# huggingface의 datasets.Dataset을 사용하는 방식

dataset = DatasetDict({
    'train' : Dataset.from_pandas(train),
    'test' : Dataset.from_pandas(test)
})

# preprocessing

def tokenize_fn(ex):
    return tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize_fn, batched=True)
dataset = dataset.rename_column("label_id", "label")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Map: 100%|██████████| 7973/7973 [00:00<00:00, 11647.74 examples/s]


In [9]:
dataset['train']

Dataset({
    features: ['media_cn_cleaned', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 31890
})

In [10]:
# setting model

num_labels = data['label_id'].nunique()
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    use_safetensors=True # gpu 버전 사용 시 추가
)

# torch gpu버전을 pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121로 설치
# huggingface transformer 라이브러리가 pytorch 2.6 미만 버전은 보안 이슈로 강제로 차단하는데, gpu 버전의 torch는 현재 pip로 설치가 불가능. 따라서 우회하는 user_safetensor를 추가

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# setting trainer
# !pip install accelerate>=0.26.0

args = TrainingArguments(
    output_dir="finetune-bert-kor",
    eval_strategy="epoch", # evaluation_strategy -> eval_strategy
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=1
)

In [12]:
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.047000,1.023184,0.719303
2,0.824000,0.979414,0.731720
3,0.632400,0.990934,0.732096


TrainOutput(global_step=5982, training_loss=0.902096227035599, metrics={'train_runtime': 2742.3864, 'train_samples_per_second': 34.886, 'train_steps_per_second': 2.181, 'total_flos': 6294314713052160.0, 'train_loss': 0.902096227035599, 'epoch': 3.0})

##### Using Torch

In [ ]:
# # huggingface가 아니라 pytorch library 사용 시 방법
# from torch.utils.data import Dataset, DataLoader

# class TokenDataset(Dataset):
  
#     def __init__(self, dataframe, model_name):
#         # sentence, label 컬럼으로 구성된 데이터프레임 전달
#         self.data = dataframe        
#         # Huggingface 토크나이저 생성
#         # self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
  
#     def __len__(self):
#         return len(self.data)
  
#     def __getitem__(self, idx):
#         sentence = self.data.iloc[idx]['document']
#         label = self.data.iloc[idx]['label']

#         # 토큰화 처리
#         tokens = self.tokenizer(
#             sentence,                # 1개 문장 
#             return_tensors='pt',     # 텐서로 반환
#             truncation=True,         # 잘라내기 적용
#             padding='max_length',    # 패딩 적용
#             add_special_tokens=True  # 스페셜 토큰 적용
#         )

#         input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
#         attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
#         token_type_ids = torch.zeros_like(attention_mask)

#         # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
#         # input_ids: 토큰
#         # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
#         # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask, 
#             'token_type_ids': token_type_ids,
#         }, torch.tensor(label)

In [ ]:
# # train, test 데이터셋 생성
# train_data = TokenDataset(train, model_name)
# test_data = TokenDataset(test, model_name)

# # DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
# train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
# test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)

In [ ]:
# # 1개의 batch 꺼내기
# inputs, labels = next(iter(train_loader))

# # 데이터셋을 device 설정
# inputs = {k: v.to(device) for k, v in inputs.items()}
# labels.to(device)